<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# Define a search space for network architectures
def sample_architecture():
    num_layers = np.random.choice([2, 3, 4])
    layer_sizes = [np.random.choice([64, 128, 256]) for _ in range(num_layers)]
    return layer_sizes

# Define a network based on the sampled architecture
class SimpleNet(nn.Module):
    def __init__(self, layer_sizes):
        super(SimpleNet, self).__init__()
        layers = []
        input_size = 28 * 28
        for size in layer_sizes:
            layers.append(nn.Linear(input_size, size))
            layers.append(nn.ReLU())
            input_size = size
        layers.append(nn.Linear(input_size, 10))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x.view(x.size(0), -1))

# Training function for the network
def train_net(net, train_loader, num_epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Testing function for the network
def test_net(net, test_loader):
    net.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return accuracy

# Load dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

best_architecture = None
best_accuracy = 0

for _ in range(10):  # Randomly sample 10 architectures
    architecture = sample_architecture()
    net = SimpleNet(architecture)
    train_net(net, train_loader)
    accuracy = test_net(net, test_loader)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_architecture = architecture

print(f"Best Architecture: {best_architecture}, Accuracy: {best_accuracy}")